# Rf 5 1 0_Wsnamedsets
Organization and simultaneous fits: working with named parameter sets and parameter snapshots in workspaces



**Author:**  04/2009 - Wouter Verkerke  
<i><small>This notebook tutorial was automatically generated with <a href= "https://github.com/root-project/root/blob/master/documentation/doxygen/converttonotebook.py">ROOTBOOK-izer</a> from the macro found in the ROOT repository  on Thursday, August 29, 2019 at 02:56 AM.</small></i>

In [1]:
%%cpp -d
#include "RooRealVar.h"
#include "RooDataSet.h"
#include "RooGaussian.h"
#include "RooConstVar.h"
#include "RooChebychev.h"
#include "RooAddPdf.h"
#include "RooWorkspace.h"
#include "RooPlot.h"
#include "TCanvas.h"
#include "TAxis.h"
#include "TFile.h"
#include "TH1.h"

In [2]:
%%cpp -d
// This is a workaround to make sure the namespace is used inside functions
using namespace RooFit;

In [3]:
void fillWorkspace(RooWorkspace &w);

 A helper function is created: 

In [4]:
%%cpp -d
void fillWorkspace(RooWorkspace &w)
{
   // C r e a t e   m o d e l
   // -----------------------

   // Declare observable x
   RooRealVar x("x", "x", 0, 10);

   // Create two Gaussian PDFs g1(x,mean1,sigma) anf g2(x,mean2,sigma) and their parameters
   RooRealVar mean("mean", "mean of gaussians", 5, 0, 10);
   RooRealVar sigma1("sigma1", "width of gaussians", 0.5);
   RooRealVar sigma2("sigma2", "width of gaussians", 1);

   RooGaussian sig1("sig1", "Signal component 1", x, mean, sigma1);
   RooGaussian sig2("sig2", "Signal component 2", x, mean, sigma2);

   // Build Chebychev polynomial p.d.f.
   RooRealVar a0("a0", "a0", 0.5, 0., 1.);
   RooRealVar a1("a1", "a1", 0.2, 0., 1.);
   RooChebychev bkg("bkg", "Background", x, RooArgSet(a0, a1));

   // Sum the signal components into a composite signal p.d.f.
   RooRealVar sig1frac("sig1frac", "fraction of component 1 in signal", 0.8, 0., 1.);
   RooAddPdf sig("sig", "Signal", RooArgList(sig1, sig2), sig1frac);

   // Sum the composite signal and background
   RooRealVar bkgfrac("bkgfrac", "fraction of background", 0.5, 0., 1.);
   RooAddPdf model("model", "g1+g2+a", RooArgList(bkg, sig), bkgfrac);

   // Import model into p.d.f.
   w.import(model);

   // E n c o d e   d e f i n i t i o n   o f   p a r a m e t e r s   i n   w o r k s p a c e
   // ---------------------------------------------------------------------------------------

   // Define named sets "parameters" and "observables", which list which variables should be considered
   // parameters and observables by the users convention
   //
   // Variables appearing in sets _must_ live in the workspace already, or the autoImport flag
   // of defineSet must be set to import them on the fly. Named sets contain only references
   // to the original variables, therefore the value of observables in named sets already
   // reflect their 'current' value
   RooArgSet *params = (RooArgSet *)model.getParameters(x);
   w.defineSet("parameters", *params);
   w.defineSet("observables", x);

   // E n c o d e   r e f e r e n c e   v a l u e   f o r   p a r a m e t e r s   i n   w o r k s p a c e
   // ---------------------------------------------------------------------------------------------------

   // Define a parameter 'snapshot' in the p.d.f.
   // Unlike a named set, a parameter snapshot stores an independent set of values for
   // a given set of variables in the workspace. The values can be stored and reloaded
   // into the workspace variable objects using the loadSnapshot() and saveSnapshot()
   // methods. A snapshot saves the value of each variable, any errors that are stored
   // with it as well as the 'Constant' flag that is used in fits to determine if a
   // parameter is kept fixed or not.

   // Do a dummy fit to a (supposedly) reference dataset here and store the results
   // of that fit into a snapshot
   RooDataSet *refData = model.generate(x, 10000);
   model.fitTo(*refData, PrintLevel(-1));

   // The kTRUE flag imports the values of the objects in (*params) into the workspace
   // If not set, the present values of the workspace parameters objects are stored
   w.saveSnapshot("reference_fit", *params, kTRUE);

   // Make another fit with the signal component forced to zero
   // and save those parameters too

   bkgfrac.setVal(1);
   bkgfrac.setConstant(kTRUE);
   bkgfrac.removeError();
   model.fitTo(*refData, PrintLevel(-1));

   w.saveSnapshot("reference_fit_bkgonly", *params, kTRUE);
}

Create model and dataset
 -----------------------------------------------

In [5]:
RooWorkspace *w = new RooWorkspace("w");
fillWorkspace(*w);


RooFit v3.60 -- Developed by Wouter Verkerke and David Kirkby 
                Copyright (C) 2000-2013 NIKHEF, University of California & Stanford University
                All rights reserved, please read http://roofit.sourceforge.net/license.txt

[#1] INFO:ObjectHandling -- RooWorkspace::import(w) importing RooAddPdf::model
[#1] INFO:ObjectHandling -- RooWorkspace::import(w) importing RooChebychev::bkg
[#1] INFO:ObjectHandling -- RooWorkspace::import(w) importing RooRealVar::x
[#1] INFO:ObjectHandling -- RooWorkspace::import(w) importing RooRealVar::a0
[#1] INFO:ObjectHandling -- RooWorkspace::import(w) importing RooRealVar::a1
[#1] INFO:ObjectHandling -- RooWorkspace::import(w) importing RooRealVar::bkgfrac
[#1] INFO:ObjectHandling -- RooWorkspace::import(w) importing RooAddPdf::sig
[#1] INFO:ObjectHandling -- RooWorkspace::import(w) importing RooGaussian::sig1
[#1] INFO:ObjectHandling -- RooWorkspace::import(w) importing RooRealVar::mean
[#1] INFO:ObjectHandling -- RooWorkspace::

Exploit convention encoded in named set "parameters" and "observables"
 to use workspace contents w/o need for introspected

In [6]:
RooAbsPdf *model = w->pdf("model");

Generate data from p.d.f. in given observables

In [7]:
RooDataSet *data = model->generate(*w->set("observables"), 1000);

Fit model to data

In [8]:
model->fitTo(*data);

[#1] INFO:Minization -- createNLL: caching constraint set under name CONSTR_OF_PDF_model_FOR_OBS_x with 0 entries
[#1] INFO:Minization -- RooMinimizer::optimizeConst: activating const optimization
[#1] INFO:Minization --  The following expressions will be evaluated in cache-and-track mode: (bkg,sig1,sig2)
 **********
 **    1 **SET PRINT           1
 **********
 **********
 **    2 **SET NOGRAD
 **********
 PARAMETER DEFINITIONS:
    NO.   NAME         VALUE      STEP SIZE      LIMITS
     1 a0           5.00000e-01  1.00000e-01    0.00000e+00  1.00000e+00
     2 a1           2.00000e-01  1.00000e-01    0.00000e+00  1.00000e+00
     3 bkgfrac      5.00000e-01  1.00000e-01    0.00000e+00  1.00000e+00
     4 mean         5.00000e+00  1.00000e+00    0.00000e+00  1.00000e+01
     5 sig1frac     8.00000e-01  1.00000e-01    0.00000e+00  1.00000e+00
 **********
 **    3 **SET ERR         0.5
 **********
 **********
 **    4 **SET PRINT           1
 **********
 **********
 **    5 **SET STR   

Plot fitted model and data on frame of first (only) observable

In [9]:
RooPlot *frame = ((RooRealVar *)w->set("observables")->first())->frame();
data->plotOn(frame);
model->plotOn(frame);

Overlay plot with model with reference parameters as stored in snapshots

In [10]:
w->loadSnapshot("reference_fit");
model->plotOn(frame, LineColor(kRed));
w->loadSnapshot("reference_fit_bkgonly");
model->plotOn(frame, LineColor(kRed), LineStyle(kDashed));

Draw the frame on the canvas

In [11]:
new TCanvas("rf510_wsnamedsets", "rf503_wsnamedsets", 600, 600);
gPad->SetLeftMargin(0.15);
frame->GetYaxis()->SetTitleOffset(1.4);
frame->Draw();

Print workspace contents

In [12]:
w->Print();


RooWorkspace(w) w contents

variables
---------
(a0,a1,bkgfrac,mean,sig1frac,sigma1,sigma2,x)

p.d.f.s
-------
RooChebychev::bkg[ x=x coefList=(a0,a1) ] = 1
RooAddPdf::model[ bkgfrac * bkg + [%] * sig ] = 1
RooAddPdf::sig[ sig1frac * sig1 + [%] * sig2 ] = 0.0106966
RooGaussian::sig1[ x=x mean=mean sigma=sigma1 ] = 1.19649e-05
RooGaussian::sig2[ x=x mean=mean sigma=sigma2 ] = 0.0588136

parameter snapshots
-------------------
reference_fit = (a0=0.500958 +/- 0.0231941,a1=0.160483 +/- 0.0372743,bkgfrac=0.504708 +/- 0.0113925,mean=5.01893 +/- 0.0101204,sigma1=0.5[C],sig1frac=0.818293 +/- 0.0374314,sigma2=1[C])
reference_fit_bkgonly = (a0=0.474267 +/- 0.0211215,a1=2.86676e-12 +/- 0.000176662,bkgfrac=1[C],mean=2.6195 +/- 2.39943,sigma1=0.5[C],sig1frac=0.818293 +/- 0.324473,sigma2=1[C])

named sets
----------
observables:(x)
parameters:(a0,a1,bkgfrac,mean,sig1frac,sigma1,sigma2)



Workspace will remain in memory after macro finishes

In [13]:
gDirectory->Add(w);

Draw all canvases 

In [14]:
%jsroot on
gROOT->GetListOfCanvases()->Draw()